# 🎛️ Professional Mixing System (Ultra Clean)

**Advanced multi-channel mixing with AI analysis and professional processing:**
- 🤖 AI-powered mix analysis and optimization
- 🎚️ Professional channel strip processing
- 🔄 Sidechain compression and frequency slotting
- 📊 Reference mix matching capabilities
- ⚡ Multiple mix configuration generation

**Features:**
- Manual balance control with group adjustments
- Intelligent frequency conflict resolution
- Professional dynamics processing
- Automated stem generation

## ⚙️ User Configuration
**Edit these settings, then run all cells**

In [1]:
# ============================================================================
# 🎛️ USER CONFIGURATION - MODIFY ALL YOUR SETTINGS HERE
# ============================================================================

# 📁 PATH SETTINGS
CHANNEL_BASE_PATH = "/Users/itay/Documents/post_mix_data/pre_mix_channels/combined_inst/"

# 🎯 REFERENCE MIX SETTINGS (Optional)
REFERENCE_MIX_PATH = "/Users/itay/Documents/post_mix_data/Reference_mix/mix/ITAY - CRASHING v.6 MIX ONLY.wav"
REFERENCE_STEMS = {
    'drums': "/Users/itay/Documents/post_mix_data/Reference_mix/stems/drums.wav",
    'bass': "/Users/itay/Documents/post_mix_data/Reference_mix/stems/bass.wav",
    'vocals': "/Users/itay/Documents/post_mix_data/Reference_mix/stems/vocals.wav",
    'music': "/Users/itay/Documents/post_mix_data/Reference_mix/stems/music.wav",
}

# 🎚️ MANUAL BALANCE CONTROL
MANUAL_CHANNEL_BALANCE = {
    # DRUMS
    'drums.kick': 2.5,
    'drums.snare': 2.5,
    'drums.hihat': 1.0,
    'drums.tom': 3.0,
    'drums.cymbal': 1.0,
    
    # VOCALS
    'vocals.lead_vocal1': 0.6,
    'vocals.lead_vocal2': 0.6,
    'vocals.lead_vocal3': 0.6,
    
    # BACKING VOCALS
    'backvocals.backing_vocal': 0.5,
    'backvocals.lead_vocal1': 0.5,
    'backvocals.lead_vocal2': 0.5,
    'backvocals.lead_vocal3': 0.5,
    'backvocals.lead_vocal4': 1.0,
    
    # BASS
    'bass.bass_guitar5': 0.5,
    'bass.bass1': 0.2,
    'bass.bass_guitar3': 0.7,
    'bass.bass_synth2': 2.5,
    'bass.bass_synth4': 1.0,
    
    # GUITARS
    'guitars.electric_guitar2': 0.4,
    'guitars.electric_guitar3': 0.1,
    'guitars.electric_guitar4': 0.4,
    'guitars.electric_guitar5': 0.4,
    'guitars.electric_guitar6': 0.1,
    'guitars.acoustic_guitar1': 0.8,
    
    # KEYS
    'keys.bell3': 0.8,
    'keys.clavinet1': 0.6,
    'keys.piano2': 0.8,
    'keys.piano4': 0.8,
    
    # SYNTHS
    'synths.pad2': 0.8,
    'synths.pad3': 0.5,
    'synths.rythmic_synth1': 0.1,
    
    # FX
    'fx.perc6': 1.2,
    'fx.fx1': 0.8,
    'fx.fx2': 0.9,
    'fx.fx3': 0.7,
    'fx.fx4': 0.8,
    'fx.fx5': 0.6,
}

# 🎚️ GROUP ADJUSTMENTS (percentage change)
GROUP_ADJUSTMENTS = {
    'drums': -30,
    'bass': -30,
    'vocals': -40,
    'guitars': 0,
    'keys': -20,
    'synths': -30,
    'fx': -60,
    'backvocals': -60,
}

# Additional global adjustment to everything except vocals
GLOBAL_ADJUSTMENT_EXCEPT_VOCALS = -10

# 🎛️ PROCESSING OPTIONS
NUM_CONFIGURATIONS = 10  # 0 = single mix only, 1-20 = multiple variants
USE_AI_ANALYSIS = True
USE_REFERENCE_MATCHING = True
USE_FIXED_ENGINE = True  # Use improved "less is more" engine
CREATE_STEMS = True
APPLY_MASTERING = True

# 🎧 OUTPUT SETTINGS
OUTPUT_BASE_DIR = "/Users/itay/Documents/post_mix_data"
PROJECT_NAME = "professional_mix"

print("✅ Configuration loaded!")
print(f"📁 Channels: {CHANNEL_BASE_PATH}")
print(f"🎯 Reference: {'Set' if REFERENCE_MIX_PATH else 'Not set'}")
print(f"🔢 Configurations: {NUM_CONFIGURATIONS if NUM_CONFIGURATIONS > 0 else 'Single mix only'}")
print(f"🤖 AI Analysis: {'Enabled' if USE_AI_ANALYSIS else 'Disabled'}")

✅ Configuration loaded!
📁 Channels: /Users/itay/Documents/post_mix_data/pre_mix_channels/combined_inst/
🎯 Reference: Set
🔢 Configurations: 10
🤖 AI Analysis: Enabled


## 📦 Load Dependencies

In [2]:
# Core imports
import os
import json
import numpy as np
import soundfile as sf
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Project modules
from pro_mixing_engine_fixed import FixedProMixingSession
from mix_intelligence import AutoMixer, MixAnalyzer
from reference_matcher import ReferenceMatcher

print("🎛️ Professional Mixing System Loaded!")
print("✅ All advanced processing capabilities available")
print("🚀 Ready for world-class mixing!")

DSP Primitives Layer loaded:
- Gain/level: apply_gain_db, normalize_peak, normalize_lufs, measure_peak, measure_rms
- Filters: highpass_filter, lowpass_filter, bandpass_filter, shelf_filter, peaking_eq, notch_filter, tilt_eq
- Stereo: mid_side_encode, mid_side_decode, stereo_widener
- Dynamics: compressor (soft‑knee), transient_shaper
- Fades: fade_in, fade_out
- K‑weighting/LUFS approx: k_weight, lufs_integrated_approx
🎛️ Professional Mixing System Loaded!
✅ All advanced processing capabilities available
🚀 Ready for world-class mixing!


## 📁 Load Channels

In [3]:
def load_channels_pro(base_path: str) -> dict:
    """Load channels for professional mixing"""
    channels = {}
    base = Path(base_path)
    
    print(f"🔍 Loading channels from: {base_path}")
    
    if not base.exists():
        print(f"❌ Directory not found: {base_path}")
        return {}
    
    # Expected categories
    categories = ['drums', 'bass', 'guitars', 'keys', 'vocals', 'backvocals', 'synths', 'strings', 'brass', 'percussion', 'fx']
    
    for category in categories:
        category_path = base / category
        if category_path.exists():
            channels[category] = {}
            audio_files = list(category_path.glob('*.wav'))
            if audio_files:
                print(f"  ✅ {category}: {len(audio_files)} files loaded")
                for audio_file in audio_files:
                    channel_name = audio_file.stem
                    channels[category][channel_name] = str(audio_file)
    
    total_channels = sum(len(tracks) for tracks in channels.values())
    print(f"\n🎚️ Total channels loaded: {total_channels}")
    print("✅ Ready for professional processing!")
    
    return channels

# Load channels from configured path
channels = load_channels_pro(CHANNEL_BASE_PATH)

🔍 Loading channels from: /Users/itay/Documents/post_mix_data/pre_mix_channels/combined_inst/
  ✅ drums: 5 files loaded
  ✅ bass: 5 files loaded
  ✅ guitars: 6 files loaded
  ✅ keys: 4 files loaded
  ✅ vocals: 3 files loaded
  ✅ backvocals: 5 files loaded
  ✅ synths: 3 files loaded
  ✅ fx: 6 files loaded

🎚️ Total channels loaded: 37
✅ Ready for professional processing!


## 🤖 AI Analysis & Mix Intelligence

In [4]:
# AI Analysis
ai_recommendations = None

if channels and USE_AI_ANALYSIS:
    print("🤖 Loading channels for AI analysis...")
    
    channel_audio = {}
    for category, tracks in channels.items():
        for name, path in tracks.items():
            try:
                audio, sr = sf.read(path)
                channel_id = f"{category}.{name}"
                channel_audio[channel_id] = audio
            except Exception as e:
                print(f"⚠️ Could not load {name}: {e}")
    
    print(f"✅ {len(channel_audio)} channels loaded for analysis")
    
    # Run AI analysis
    auto_mixer = AutoMixer(sr=44100)
    ai_recommendations = auto_mixer.auto_mix(channel_audio)
    
    print("\n🎯 AI RECOMMENDATIONS:")
    print("=" * 40)
    
    conflicts = ai_recommendations['conflicts_detected']
    if conflicts:
        print(f"🔍 Frequency conflicts detected: {len(conflicts)}")
        for i, (conflict, data) in enumerate(conflicts.items()):
            if i < 5:  # Show first 5 conflicts
                print(f"  • {conflict}: {data['severity']} conflict")
        if len(conflicts) > 5:
            print(f"  ... and {len(conflicts) - 5} more")
    
    optimal_gains = ai_recommendations['optimal_gains']
    print(f"\n📊 AI calculated optimal levels for {len(optimal_gains)} channels")

elif not USE_AI_ANALYSIS:
    print("ℹ️ AI analysis disabled in configuration")
else:
    print("❌ No channels loaded - skipping AI analysis")

🤖 Loading channels for AI analysis...
✅ 37 channels loaded for analysis
🤖 AI Auto-Mixing in progress...
  Analyzing channels...
  Detecting frequency conflicts...
  Calculating optimal balance...
  Generating processing recommendations...
  Resolving frequency conflicts...

🎯 AI RECOMMENDATIONS:
🔍 Frequency conflicts detected: 15
  • drums.tom_vs_drums.kick: medium conflict
  • drums.tom_vs_bass.bass_guitar5: medium conflict
  • drums.tom_vs_bass.bass1: medium conflict
  • drums.tom_vs_bass.bass_guitar3: medium conflict
  • drums.tom_vs_bass.bass_synth4: high conflict
  ... and 10 more

📊 AI calculated optimal levels for 37 channels


## 🎛️ Create Professional Mix Session

In [5]:
if channels:
    print("🎛️ Creating professional mixing session...")
    
    # Create session with fixed engine
    pro_session = FixedProMixingSession(
        channels=channels,
        sample_rate=44100
    )
    
    print(f"✅ Session created with {len(pro_session.channel_strips)} channels")
    
    # Apply professional processing
    print("\n🔧 Configuring professional processing...")
    pro_session.apply_sidechain_compression()
    print("  ✅ Sidechain compression configured")
    
    # Apply frequency slotting
    pro_session.apply_frequency_slotting()
    
    # Apply AI recommendations
    if ai_recommendations:
        print("\n🤖 Applying AI recommendations...")
        
        # Apply optimal gains (scaled for musicality)
        for ch_id, strip in pro_session.channel_strips.items():
            if ch_id in ai_recommendations['optimal_gains']:
                ai_gain = ai_recommendations['optimal_gains'][ch_id]
                # Scale down extreme gains
                if ai_gain > 5:
                    ai_gain = 2 + (ai_gain - 2) * 0.3
                elif ai_gain < 0.2:
                    ai_gain = 0.2 + (ai_gain - 0.2) * 0.5
                
                strip.gain *= ai_gain
        
        # Apply gentle EQ for conflicts
        eq_solutions = ai_recommendations['eq_solutions']
        for solution_name, solution in eq_solutions.items():
            channel = solution['channel']
            for ch_id, strip in pro_session.channel_strips.items():
                if channel in ch_id:
                    gain = solution['gain'] * 0.5  # Half the suggested amount
                    if abs(gain) > 0.5:
                        eq_band = {
                            'freq': solution['freq'],
                            'gain': gain,
                            'q': solution['q'] * 1.2,
                        }
                        strip.eq_bands.append(eq_band)
                    break
        
        print("  ✅ AI recommendations applied musically")
    
    print("\n🎵 Professional mixing session ready!")

else:
    print("❌ Cannot create session - no channels loaded")

🎛️ Creating professional mixing session...
🎵 Loading channels with musical processing...
  ✓ drums.tom loaded
  ✓ drums.hihat loaded
  ✓ drums.kick loaded
  ✓ drums.snare loaded
  ✓ drums.cymbal loaded
  ✓ bass.bass_guitar5 loaded
  ✓ bass.bass1 loaded
  ✓ bass.bass_guitar3 loaded
  ✓ bass.bass_synth2 loaded
  ✓ bass.bass_synth4 loaded
  ✓ guitars.electric_guitar4 loaded
  ✓ guitars.electric_guitar5 loaded
  ✓ guitars.electric_guitar6 loaded
  ✓ guitars.electric_guitar2 loaded
  ✓ guitars.acoustic_guitar1 loaded
  ✓ guitars.electric_guitar3 loaded
  ✓ keys.bell3 loaded
  ✓ keys.clavinet1 loaded
  ✓ keys.piano4 loaded
  ✓ keys.piano2 loaded
  ✓ vocals.lead_vocal3 loaded
  ✓ vocals.lead_vocal2 loaded
  ✓ vocals.lead_vocal1 loaded
  ✓ backvocals.lead_vocal3 loaded
  ✓ backvocals.lead_vocal2 loaded
  ✓ backvocals.backing_vocal loaded
  ✓ backvocals.lead_vocal1 loaded
  ✓ backvocals.lead_vocal4 loaded
  ✓ synths.rythmic_synth1 loaded
  ✓ synths.pad3 loaded
  ✓ synths.pad2 loaded
  ✓ fx.perc

## 🎯 Reference Mix Matching (Optional)

In [6]:
if USE_REFERENCE_MATCHING and REFERENCE_MIX_PATH and 'pro_session' in locals():
    print("🎯 REFERENCE MIX ANALYSIS & MATCHING")
    print("=" * 50)
    
    # Check if stems are provided
    stem_paths = {k: v for k, v in REFERENCE_STEMS.items() if v and os.path.exists(v)}
    
    if stem_paths:
        print(f"🎛️ Using {len(stem_paths)} reference stems for precise matching!")
    else:
        print("🎵 Using full mix analysis")
    
    try:
        # Initialize reference matcher
        matcher = ReferenceMatcher()
        
        # Analyze reference and match our mix to it
        matching_results = matcher.analyze_and_match(
            REFERENCE_MIX_PATH, 
            pro_session,
            stem_paths if stem_paths else None
        )
        
        print(f"\n✅ REFERENCE ANALYSIS COMPLETE!")
        
        # Show reference characteristics
        ref_analysis = matching_results['reference_analysis']
        print(f"\n📈 REFERENCE MIX CHARACTERISTICS:")
        print(f"  • Loudness: {ref_analysis.loudness_lufs:.1f} LUFS")
        print(f"  • Dynamic Range: {ref_analysis.dynamic_range:.1f} dB") 
        print(f"  • Stereo Width: {ref_analysis.stereo_width:.2f}")
        print(f"  • Punch Factor: {ref_analysis.punch_factor:.2f}")
        
        print(f"\n🎛️ Mix adjusted to match reference characteristics")
        
    except FileNotFoundError:
        print(f"❌ Reference file not found: {REFERENCE_MIX_PATH}")
        
    except Exception as e:
        print(f"⚠️ Reference analysis failed: {e}")
        print("   Continuing without reference matching...")

elif USE_REFERENCE_MATCHING and not REFERENCE_MIX_PATH:
    print("ℹ️ Reference matching enabled but no path provided")
    
else:
    print("ℹ️ Reference matching disabled in configuration")

🎯 REFERENCE MIX ANALYSIS & MATCHING
🎛️ Using 4 reference stems for precise matching!
🎯 REFERENCE MATCHING SYSTEM
⚠️ Reference analysis failed: 'ReferenceMixAnalyzer' object has no attribute 'analyze_reference'
   Continuing without reference matching...


## 🎚️ Apply Manual Balance Control

In [7]:
if 'pro_session' in locals():
    print("🎚️ APPLYING MANUAL BALANCE CONTROL")
    print("=" * 40)
    
    # Helper function for percentage adjustments
    def apply_percentage_adjustment(overrides, percent, group=None):
        """Apply percentage change to a group"""
        if percent == 0:
            return overrides
        
        factor = 1 + percent / 100.0
        for channel, vol in overrides.items():
            prefix = channel.split('.')[0]
            if group and prefix == group:
                overrides[channel] = vol * factor
        return overrides
    
    # Start with configured balance
    manual_overrides = MANUAL_CHANNEL_BALANCE.copy()
    
    # Apply group adjustments
    for group, percent in GROUP_ADJUSTMENTS.items():
        if percent != 0:
            manual_overrides = apply_percentage_adjustment(manual_overrides, percent, group)
            print(f"  • {group}: {percent:+d}% adjustment applied")
    
    # Apply global adjustment (except vocals)
    if GLOBAL_ADJUSTMENT_EXCEPT_VOCALS != 0:
        for channel, vol in manual_overrides.items():
            if 'vocal' not in channel.lower():
                manual_overrides[channel] = vol * (1 + GLOBAL_ADJUSTMENT_EXCEPT_VOCALS / 100.0)
        print(f"  • Global (except vocals): {GLOBAL_ADJUSTMENT_EXCEPT_VOCALS:+d}% adjustment")
    
    # Apply to session
    applied_count = 0
    for ch_id, strip in pro_session.channel_strips.items():
        if ch_id in manual_overrides:
            manual_gain = manual_overrides[ch_id]
            strip.gain = manual_gain
            applied_count += 1
    
    print(f"\n✅ Applied balance to {applied_count} channels")
    
    # Show summary of extremes
    very_loud = [ch for ch, gain in manual_overrides.items() if gain > 2.5]
    very_quiet = [ch for ch, gain in manual_overrides.items() if gain < 0.3]
    
    if very_loud:
        print(f"📢 Boosted channels: {', '.join(very_loud)}")
    if very_quiet:
        print(f"🔇 Reduced channels: {', '.join(very_quiet)}")

else:
    print("❌ No professional session available")

🎚️ APPLYING MANUAL BALANCE CONTROL
  • drums: -30% adjustment applied
  • bass: -30% adjustment applied
  • vocals: -40% adjustment applied
  • keys: -20% adjustment applied
  • synths: -30% adjustment applied
  • fx: -60% adjustment applied
  • backvocals: -60% adjustment applied
  • Global (except vocals): -10% adjustment

✅ Applied balance to 37 channels
🔇 Reduced channels: backvocals.backing_vocal, backvocals.lead_vocal1, backvocals.lead_vocal2, backvocals.lead_vocal3, bass.bass1, guitars.electric_guitar3, guitars.electric_guitar6, synths.rythmic_synth1, fx.fx1, fx.fx3, fx.fx4, fx.fx5


## 🚀 Process Professional Mix

In [8]:
if 'pro_session' in locals():
    # Create output directory
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    if NUM_CONFIGURATIONS == 0:
        # Single mix
        output_dir = os.path.join(OUTPUT_BASE_DIR, f"professional_mixes/mix_{timestamp}")
        print(f"🎛️ PROCESSING SINGLE PROFESSIONAL MIX")
        print("=" * 50)
        
    else:
        # Multiple configurations
        base_output_dir = os.path.join(OUTPUT_BASE_DIR, f"professional_mixes/{NUM_CONFIGURATIONS}_configurations_{timestamp}")
        print(f"🎛️ PROCESSING {NUM_CONFIGURATIONS} MIX CONFIGURATIONS")
        print("=" * 50)
    
    print(f"📁 Output: {output_dir if NUM_CONFIGURATIONS == 0 else base_output_dir}")
    print("\\n🔄 Processing with professional algorithms...")
    print("  • Gentle sidechain compression (kick → bass)")
    print("  • Musical parallel compression on drums")
    print("  • Minimal saturation for warmth")
    print("  • Natural multiband processing")
    print("  • Transparent mastering chain")
    print("\\n⏳ This may take a few minutes...")
    
    try:
        if NUM_CONFIGURATIONS == 0:
            # Process single mix
            pro_results = pro_session.process_mix(output_dir)
            
            print("\\n" + "=" * 50)
            print("🏆 PROFESSIONAL MIX COMPLETE!")
            print("=" * 50)
            print(f"📊 FINAL RESULTS:")
            print(f"  • Peak Level: {pro_results['peak_db']:.1f} dBFS")
            print(f"  • RMS Level: {pro_results['rms_db']:.1f} dBFS")
            print(f"  • Processing Time: {pro_results['time']:.1f} seconds")
            print(f"  • Output File: {pro_results['output_file']}")
            
            # Analyze final mix
            print("\\n🔍 Analyzing final mix quality...")
            final_mix, sr = sf.read(pro_results['output_file'])
            analyzer = MixAnalyzer(sr)
            analysis = analyzer.analyze_full_mix(final_mix)
            
            print("\\n📈 MIX ANALYSIS:")
            print("-" * 30)
            print(f"  LUFS Loudness: {analysis.loudness_lufs:.1f} (target: -14 LUFS)")
            print(f"  Dynamic Range: {analysis.dynamic_range:.1f} dB")
            print(f"  Stereo Width: {analysis.stereo_width:.2f}")
            print(f"  Phase Correlation: {analysis.phase_correlation:.3f}")
            
            if analysis.issues:
                print("\\n⚠️ POTENTIAL ISSUES:")
                for issue in analysis.issues:
                    print(f"  • {issue}")
            else:
                print("\\n✅ No significant issues detected!")
                
        else:
            # Process multiple configurations with meaningful names
            if hasattr(pro_session, '_get_mix_configurations'):
                # Get available configurations
                all_configs = pro_session._get_mix_configurations()
                config_list = list(all_configs.items())
                
                # Select requested number
                actual_num = min(NUM_CONFIGURATIONS, len(config_list))
                selected_configs = dict(config_list[:actual_num])
                
                print(f"📋 Processing {actual_num} configuration(s)")
                print(f"⏳ Estimated time: {actual_num * 30} seconds")
                
                # Process each configuration
                for i, (config_name, config_data) in enumerate(selected_configs.items(), 1):
                    print(f"\\n[{i}/{actual_num}] {config_name}")
                    print(f"   {config_data['description']}")
                    
                    config_dir = os.path.join(base_output_dir, f"{i:02d}_{config_name}")
                    
                    try:
                        # Apply configuration
                        pro_session.apply_mix_configuration(config_name)
                        
                        # Process mix
                        result = pro_session.process_mix(config_dir)
                        print(f"  ✅ Complete")
                        
                    except Exception as e:
                        print(f"  ❌ Failed: {e}")
                
                print(f"\\n✅ All configurations saved in: {base_output_dir}")
                
            else:
                # Fallback to generic variations
                actual_num = min(NUM_CONFIGURATIONS, 10)
                print(f"📋 Processing {actual_num} variation(s)")
                
                for i in range(1, actual_num + 1):
                    print(f"\\n[{i}/{actual_num}] Processing variation {i}")
                    
                    config_dir = os.path.join(base_output_dir, f"variation_{i:02d}")
                    
                    try:
                        result = pro_session.process_mix(config_dir)
                        print(f"  ✅ Complete")
                    except Exception as e:
                        print(f"  ❌ Failed: {e}")
                
                print(f"\\n✅ All variations saved in: {base_output_dir}")
        
        print("\\n" + "=" * 50)
        print("🎉 PROFESSIONAL MIX READY!")
        print("🎧 Your mix has been processed with professional algorithms")
        print("✨ Enjoy your professional-quality mix!")
        print("=" * 50)
        
    except Exception as e:
        print(f"\\n❌ Error during mixing: {e}")
        import traceback
        traceback.print_exc()

else:
    print("❌ No professional session available - please run previous cells")

🎛️ PROCESSING 10 MIX CONFIGURATIONS
📁 Output: /Users/itay/Documents/post_mix_data/professional_mixes/10_configurations_20250901_230635
\n🔄 Processing with professional algorithms...
  • Gentle sidechain compression (kick → bass)
  • Musical parallel compression on drums
  • Minimal saturation for warmth
  • Natural multiband processing
  • Transparent mastering chain
\n⏳ This may take a few minutes...
📋 Processing 10 configuration(s)
⏳ Estimated time: 300 seconds
\n[1/10] Standard_Mix
   Balanced mix with moderate processing
🎛️ Applying configuration: Standard_Mix
   Balanced mix with moderate processing
🎚️ Processing musical mix...
  Processing drums/tom...
  Processing drums/hihat...
  Processing drums/kick...
    🔒 Modern production limiting: kick
  Processing drums/snare...


KeyboardInterrupt: 

## 🎉 Summary

In [ ]:
# Final summary
print("=" * 60)
print("🎉 PROFESSIONAL MIXING COMPLETE!")
print("=" * 60)

if 'pro_results' in locals():
    print(f"📁 Output: {os.path.dirname(pro_results['output_file'])}")
    
    if NUM_CONFIGURATIONS == 0:
        print(f"🎵 Single professional mix created")
        print(f"   File: {os.path.basename(pro_results['output_file'])}")
    else:
        print(f"🎛️ {NUM_CONFIGURATIONS} mix configurations created")
        
    if CREATE_STEMS:
        print(f"🎚️ Individual stems included")
        
elif NUM_CONFIGURATIONS > 0 and 'base_output_dir' in locals():
    print(f"📁 Output: {base_output_dir}")
    print(f"🎛️ {NUM_CONFIGURATIONS} mix configurations created")
    
print()
print("✨ Features applied:")
if USE_AI_ANALYSIS:
    print("  • AI-powered mix optimization")
if USE_REFERENCE_MATCHING:
    print("  • Reference mix matching")
print("  • Professional channel strip processing")
print("  • Intelligent frequency slotting")
print("  • Manual balance control")
if APPLY_MASTERING:
    print("  • Professional mastering chain")

print()
print("🎧 Ready for listening!")